In [1]:
!pip install datasets==2.11
!pip install datasets transformers[sentencepiece]
!pip install sacrebleu rouge_score evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [2]:
from datasets import load_dataset, load_metric
import pandas as pd
import evaluate 
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import DatasetDict
import numpy as np
import evaluate

sacrebleu_metric = load_metric("sacrebleu")
rouge_metric = evaluate.load('rouge')
ter_metric = evaluate.load("ter")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_32/3696305817.py:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  sacrebleu_metric = load_metric("sacrebleu")


In [3]:
def preprocess_dataset(data):
    conditions = [data.ref_tox < data.trn_tox ]
    values = ['true']
    data['swap'] = np.select(conditions, values)
    
    is_swap = data['swap'] == 'true'
    data.loc[is_swap, ['reference', 'translation', 'ref_tox', 'trn_tox']] = (
        data.loc[is_swap, ['translation', 'reference', 'trn_tox', 'ref_tox']].values
        )
    
    index_drop = data[(data['ref_tox'] <= 0.8) | (data['trn_tox'] >= 0.2) ].index
    data.drop(index_drop, inplace=True)
    data.drop(columns=["swap"], axis=1, inplace=True)
    return data

In [4]:
d = pd.read_table("/kaggle/input/paranmt/filtered.tsv")
d = preprocess_dataset(d)
d.to_csv("converted.csv")

In [5]:
d.head()

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
3,3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
4,4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402
5,5,I'm not gonna have a child... ...with the same...,I'm not going to breed kids with a genetic dis...,0.703185,0.206522,0.950956,0.035846


In [6]:
raw_datasets = load_dataset("csv", data_files="/kaggle/working/converted.csv")
raw_datasets = raw_datasets['train'].train_test_split(test_size=1-50000/raw_datasets.num_rows["train"], seed=42)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b03c98139713f04b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
raw_datasets.num_rows

{'train': 50000, 'test': 439290}

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 439290
    })
})

In [9]:
#model_checkpoint = "s-nlp/t5-paranmt-detox"
model_checkpoint = "s-nlp/bart-base-detox"
#model_checkpoint = "t5-base"

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate tox to detox: "
else:
    prefix = ""

In [12]:
max_input_length = 128
max_target_length = 128

def preprocess_function_extra(tokenizer, model):
    def preprocess_function(examples):
        inputs = [prefix + ex for ex in examples["reference"]]
        targets = [ex for ex in examples["translation"]]
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    return preprocess_function

In [13]:
#preprocess_function(raw_datasets['train'][:2])

NameError: name 'preprocess_function' is not defined

In [14]:
raw_datasets_train = raw_datasets['train'].train_test_split(test_size=0.3, seed=42)
raw_datasets_test = raw_datasets_train['test'].train_test_split(test_size=0.5, seed=42)


ds_splits = DatasetDict({
    'train': raw_datasets_train['train'],
    'valid': raw_datasets_test['train'],
    'test': raw_datasets_test['test']
})

In [15]:
ds_splits

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 35000
    })
    valid: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 7500
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 7500
    })
})

In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [19]:
tokenized_datasets = ds_splits.map(preprocess_function_extra(tokenizer,model), batched=True)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [20]:
source_lang = "toxic"
target_lang = "detoxic"
batch_size = 16

In [21]:
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    report_to="none"
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metric_with_extra(tokenizer):
    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)

        result = {"bleu": sacrebleu_result["score"],
                "rouge1": rouge_result["rouge1"],
                 "rouge2": rouge_result["rouge2"],
                 "TER": ter_result["score"]}
        print(result)
        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        return result
    return compute_metrics

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metric_with_extra(tokenizer)
)

In [25]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Rouge1,Rouge2,Ter,Gen Len
1,1.969300,1.805549,23.402400,0.575800,0.347500,63.943700,19.944500
2,1.783200,1.773347,23.677700,0.577700,0.350900,63.643000,19.931700


{'bleu': 23.402445155928408, 'rouge1': 0.57583991140112, 'rouge2': 0.34753189314992283, 'TER': 63.94365423691581}
{'bleu': 23.67771648137215, 'rouge1': 0.5776906677466493, 'rouge2': 0.35092409882206455, 'TER': 63.64297874677357}


TrainOutput(global_step=4376, training_loss=1.9123117701445027, metrics={'train_runtime': 1136.0219, 'train_samples_per_second': 61.619, 'train_steps_per_second': 3.852, 'total_flos': 1457413042176000.0, 'train_loss': 1.9123117701445027, 'epoch': 2.0})

In [26]:
trainer.evaluate()

{'bleu': 23.67771648137215, 'rouge1': 0.5776906677466493, 'rouge2': 0.35092409882206455, 'TER': 63.64297874677357}


{'eval_loss': 1.7733469009399414,
 'eval_bleu': 23.6777,
 'eval_rouge1': 0.5777,
 'eval_rouge2': 0.3509,
 'eval_TER': 63.643,
 'eval_gen_len': 19.9317,
 'eval_runtime': 257.7786,
 'eval_samples_per_second': 29.095,
 'eval_steps_per_second': 1.819,
 'epoch': 2.0}

In [27]:
trainer.save_model()

In [28]:
predictions, labels, metrics = trainer.predict(tokenized_datasets["test"], metric_key_prefix="predict")

{'bleu': 24.223545816197163, 'rouge1': 0.5789883810317589, 'rouge2': 0.3537359428406176, 'TER': 62.88037230432555}


In [29]:
my_predictions = tokenizer.batch_decode(
                    predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )

In [30]:
my_predictions[:15]

['Scary, seven feet tall, with sharp teeth.',
 "I'm a joke?",
 'The British... a lot of them are not welcome here.',
 'You could be a beautiful pig.',
 "that's why I keep chasing you.",
 "oh, my God, they're so gross.",
 "Since the bomb didn't explode, he must have been caught and executed.",
 'But you have to promise me that the next time you sleep with your wife, you',
 '"my daughter wanted to marry a bad guy once," Crosby said darkly.',
 "in fact, it's about the entire 2nd Mass and how you're going to",
 "he'll remove the cancerous part of the tongue and then replace it with a strip",
 "girls, let's go sit with the bullet.",
 "You're gonna strangle me?",
 'Quiet, boy.',
 "There are still some barbarians hiding on the island, but I'm sure we'll"]

In [31]:
ds_splits["test"]["reference"][:15]

['terrible, seven feet tall, with sharp teeth.',
 "I'm the fucking joke?",
 'the British...... a lot of them are not welcome here.',
 'you could be a beautiful pig.',
 "That's why I keep ducking you.",
 "God, they're so gross.",
 "since the bomb didn't explode, he must have been caught and executed.",
 'but you have to promise me that the next time you fuck your wife, you have to call you "Jay."',
 '"My daughter wanted to marry a pissant once," said Crosby darkly.',
 "In fact, it's about the entire 2nd Mass and how you're gonna crawl back to whatever dung heap you came from and leave us all alone.",
 "Uh,he'il remove the cancerous part of the tongue,and then reconstru it with a strip of flesh from your legs.",
 "Girls, let's go sit with the bullet.",
 'you gonna strangle me?',
 'shut up, boy.',
 "there are still some barbarians in hiding on the island, but I'm sure we'll find them soon, and we'll kill them, every Barbar we don't charge, will be sent by ship along with the rest of the p

In [32]:
trainer.state.log_history

[{'loss': 2.1844,
  'learning_rate': 1.7719378427787937e-05,
  'epoch': 0.23,
  'step': 500},
 {'loss': 2.0407,
  'learning_rate': 1.543875685557587e-05,
  'epoch': 0.46,
  'step': 1000},
 {'loss': 1.9981,
  'learning_rate': 1.3153564899451555e-05,
  'epoch': 0.69,
  'step': 1500},
 {'loss': 1.9693,
  'learning_rate': 1.0872943327239488e-05,
  'epoch': 0.91,
  'step': 2000},
 {'eval_loss': 1.8055486679077148,
  'eval_bleu': 23.4024,
  'eval_rouge1': 0.5758,
  'eval_rouge2': 0.3475,
  'eval_TER': 63.9437,
  'eval_gen_len': 19.9445,
  'eval_runtime': 259.0059,
  'eval_samples_per_second': 28.957,
  'eval_steps_per_second': 1.811,
  'epoch': 1.0,
  'step': 2188},
 {'loss': 1.876,
  'learning_rate': 8.587751371115174e-06,
  'epoch': 1.14,
  'step': 2500},
 {'loss': 1.7728,
  'learning_rate': 6.30255941499086e-06,
  'epoch': 1.37,
  'step': 3000},
 {'loss': 1.7884,
  'learning_rate': 4.017367458866545e-06,
  'epoch': 1.6,
  'step': 3500},
 {'loss': 1.7832,
  'learning_rate': 1.7321755027422

In [33]:
train_loss = []
eval_loss = []
step = []
eval_step = []
for log in trainer.state.log_history:
    if "loss" in log.keys():
        train_loss.append(log["loss"])
        step.append(log["step"])
    if "eval_loss" in log.keys():
        eval_loss.append(log["eval_loss"])
        eval_step.append(log["step"])

In [34]:
trainer.state.log_history

[{'loss': 2.1844,
  'learning_rate': 1.7719378427787937e-05,
  'epoch': 0.23,
  'step': 500},
 {'loss': 2.0407,
  'learning_rate': 1.543875685557587e-05,
  'epoch': 0.46,
  'step': 1000},
 {'loss': 1.9981,
  'learning_rate': 1.3153564899451555e-05,
  'epoch': 0.69,
  'step': 1500},
 {'loss': 1.9693,
  'learning_rate': 1.0872943327239488e-05,
  'epoch': 0.91,
  'step': 2000},
 {'eval_loss': 1.8055486679077148,
  'eval_bleu': 23.4024,
  'eval_rouge1': 0.5758,
  'eval_rouge2': 0.3475,
  'eval_TER': 63.9437,
  'eval_gen_len': 19.9445,
  'eval_runtime': 259.0059,
  'eval_samples_per_second': 28.957,
  'eval_steps_per_second': 1.811,
  'epoch': 1.0,
  'step': 2188},
 {'loss': 1.876,
  'learning_rate': 8.587751371115174e-06,
  'epoch': 1.14,
  'step': 2500},
 {'loss': 1.7728,
  'learning_rate': 6.30255941499086e-06,
  'epoch': 1.37,
  'step': 3000},
 {'loss': 1.7884,
  'learning_rate': 4.017367458866545e-06,
  'epoch': 1.6,
  'step': 3500},
 {'loss': 1.7832,
  'learning_rate': 1.7321755027422

In [ ]:
import matplotlib.pyplot as plt


plt.plot(step, train_loss)
plt.plot(eval_step, eval_loss)
plt.legend(["train_loss", "eval_loss"], loc ="lower right") 
plt.show()

In [36]:
inputs = tokenizer("You are nigger", return_tensors="pt").input_ids
predictions = model.generate(inputs)
tokenizer.batch_decode(
                    predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)